In [2]:
import os
import traceback
import pandas as pd
import geopandas as gpd
from geopandas.tools import sjoin
import sqlalchemy
import matplotlib.pyplot as plt
import numpy as np

user='hifld_summer'
pw='hifld_summer'
name='hifld_summer'
host='moria'
port=2023
engine = sqlalchemy.create_engine(
    f'postgresql://{user}:{pw}@{host}:{port}/{name}'
    )

In [9]:
sql="""
select geom, build_id, prim_occ from structures.alabama limit 500000
"""
try:
    gdf = gpd.GeoDataFrame.from_postgis(sql, con=engine,geom_col='geom') # epsg 4346

    gdf = gdf.to_crs("ESRI:102003") # USA_Contiguous_Albers_Equal_Area_Conic Projection

    gdf["centroid"] = gdf.geometry.centroid
    
    buffer_distance = 250
    gdf["buffer_geom"] = gdf.centroid.buffer(buffer_distance)

    buffers_gdf = gdf.copy()
    centroids_gdf = gdf.copy()
    
    buffers_gdf = buffers_gdf.set_geometry('buffer_geom')
    centroids_gdf = gdf.set_geometry('centroid')

    buffers_gdf.rename(columns = {'build_id':'build_id_right'}, inplace = True)
    centroids_gdf.rename(columns = {'build_id':'build_id_left'}, inplace = True)
    
    buffers_gdf = buffers_gdf.drop(columns=['centroid', 'geom'])
    centroids_gdf = centroids_gdf.drop(columns=['buffer_geom', 'geom'])

    def count_points_in_polygons(points, polygons, polygon_id, new_column="points_count"):

        points_in_polygon = (polygons.sjoin(points, how="inner").groupby(polygon_id).size().rename(new_column))
        polygons = (polygons.set_index(polygon_id).join(points_in_polygon, how="left").fillna({new_column: 0}))

        return polygons

    
    neighbor_gdf = count_points_in_polygons(centroids_gdf, buffers_gdf, "build_id_right", new_column="neighbor_count")

    neighbor_gdf["neighbor_count"] = neighbor_gdf["neighbor_count"] - 1

except Exception:
    print(traceback.print_exc())

count    5000.000000
mean      110.600800
std        92.038308
min         0.000000
25%        36.000000
50%        78.000000
75%       167.000000
max       365.000000
Name: neighbor_count, dtype: float64
